<hr style="color:#a4342d;">
<p align="center">
    <b style="font-size:2.5vw; color:#a4342d; font-weight:bold;">
    Introduction to machine learning - Homework 3
    </b>
</p>
<hr style="color:#a4342d;">

<b>Authors</b>: <i>C. Bosch, M. Cornet & V. Mangeleer</i>

[comment]: <> (Section)
<hr style="color:#a4342d;">
<p align="center">
    <b style="font-size:1.5vw; color:#a4342d;">
    Initialization
    </b>
</p>
<hr style="color:#a4342d;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to initialize all the librairies needed and load the untouched dataset.

In [52]:
# -- LIBRAIRIES --
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Allow notebook to plot in terminal
%matplotlib inline

In [ ]:
# -- FUNCTION --

# Used to print a basic section title in terminal
def section(title = "UNKNOWN"):

    # Number of letters to determine section size
    title_size = len(title)

    # Section title boundaries
    boundary  = "-"
    for i in range(title_size + 1):
        boundary += "-"
    
    # Printing section
    print(boundary)
    print(f" {title} ")
    print(boundary)

In [62]:
# -- ORIGINAL DATASET --
# The original dataset is contained in the "data/original" folder

# Stores the original dataset
dataset_original_X = []
dataset_original_Y = []

# Load the original dataset
for i in range(1, 11):
    dataset_original_X.append(pd.read_csv(f"data/original/X_Zone_{i}.csv"))
    dataset_original_Y.append(pd.read_csv(f"data/original/Y_Zone_{i}.csv"))

In [33]:
# -- BASIC INFORMATION DATASET --

# Loading X and Y dataset for the first wind turbine
dataset_X1 = dataset_original_X[0]
dataset_Y1 = dataset_original_Y[0]

# Displaying their relative information
section("WIND TURBINE 1 - X Dataset")
section("HEAD")
print(dataset_X1.head())
section("INFO")
dataset_X1.info()

section("WIND TURBINE 1 - Y Dataset")
section("HEAD")
print(dataset_Y1.head())
section("INFO")
dataset_Y1.info()

----------------------------
 WIND TURBINE 1 - X Dataset 
----------------------------
------
 HEAD 
------
   ZONEID       U10       V10      U100      V100  Day  Month  Year  Hour  \
0       1  0.116541 -0.253693  0.084184 -0.213789    1      1  2012     1   
1       1  0.155554 -0.162680  0.115345 -0.132693    1      1  2012     2   
2       1  0.170341 -0.062468  0.125952 -0.048266    1      1  2012     3   
3       1  0.149248  0.007347  0.106940  0.009691    1      1  2012     4   
4       1  0.128458  0.062183  0.090240  0.056152    1      1  2012     5   

   U10_mean  ...  U10_(t-1)  V10_(t-3)  V10_(t-2)  V10_(t-1)  U100_(t-3)  \
0  0.116541  ...   0.000000   0.000000   0.000000   0.000000    0.000000   
1  0.155554  ...   0.116541   0.000000   0.000000  -0.253693    0.000000   
2  0.116541  ...   0.155554   0.000000  -0.253693  -0.162680    0.000000   
3  0.136048  ...   0.170341  -0.253693  -0.162680  -0.062468    0.084184   
4  0.147479  ...   0.149248  -0.162680  -0.062468

[comment]: <> (Section)
<hr style="color:#a4342d;">
<p align="center">
    <b style="font-size:1.5vw; color:#a4342d;">
    Exploring
    </b>
</p>
<hr style="color:#a4342d;">

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to gain some basic insight regarding the dataset

In [ ]:
# -- HISTOGRAM --

# Extracting only relevant variables
dataset_X1_relevant = dataset_X1[["U10", "U100", "V10", "V100"]]
dataset_Y1_relevant = dataset_Y1[dataset_Y1["TARGETVAR"] >= 0]   # /!\ Removing test samples (y = -1) /!\
dataset_Y1_relevant = dataset_Y1_relevant[["TARGETVAR"]]

# Observing distributions
dataset_X1_relevant.hist(bins = 60, figsize = (20, 15))
dataset_Y1_relevant.hist(bins = 60, figsize = (15, 4))

In [ ]:
# -- OBSERVING WIND vs POWER --

# Removing test data
dataset_X_clean = dataset_X1[dataset_Y1["TARGETVAR"] >= 0]
dataset_Y_clean = dataset_Y1[dataset_Y1["TARGETVAR"] >= 0]

# Computing total wind speed
u_wind   = dataset_X_clean[["U100"]].to_numpy()
v_wind   = dataset_X_clean[["V100"]].to_numpy()
wind_tot = np.sqrt(u_wind**2 + v_wind**2)

# Retreiving power
power = dataset_Y_clean[["TARGETVAR"]].to_numpy()

# To see more clearly, one sample out of 2 is removed
for i in range(1):
    wind_tot = wind_tot[1::2]
    power    = power[1::2]

# Plotting
plt.figure(figsize=(15, 10))
plt.scatter(wind_tot, power, s = 3)
plt.grid()
plt.xlabel("Speed [m/s]")
plt.ylabel("Normalized Power [-]")
plt.show()

[comment]: <> (Section)
<hr style="color:#a4342d;"></hr>
<p align="center">
    <b style="font-size:1.5vw; color:#a4342d;">
    Dataset - Train & Test | DataLoader
    </b>
</p>
<hr style="color:#a4342d;"></hr>

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to explore further the dataset ! First, one needs to create a train and test set. Then, it is interesitng to look for correlations, new variables and possible improvements to the current dataset. All the new datasets will be save in the datafolder and ready to use by our different models ! The functions available are:
</p>

In [44]:
# -- FUNCTIONS : MEAN, VARIANCE, ZONAL AVERAGE SPEED AND TIME STEPS --
#
# Used to compute the mean and variance of a variable over some timeslices in the dataset
def computeMeanVariance(datasets, 
                        variables = ["U100"     , "V100"],
                        names_m   = ["U100_mean", "V100_mean"],
                        names_v   = ["U100_var" , "V100_var"],
                        window    = 100,
                        variance  = True):

    # Security
    assert len(variables) == len(names_m), "Each variable should have a name for its mean"
    assert len(variables) == len(names_v), "Each variable should have a name for its var"
    assert window > 1, "Window size must be greater than 1 to compute mean and var"

    # Looping over all the datasets
    for d in datasets:

        # Looping over the variables whose mean and var must be computed
        for v, nm, nv in zip(variables, names_m, names_v):

            # Retreiving data 
            data = d.loc[: , [v]].to_numpy()

            # Stores mean and variance (1st and 2nd : mean = their value, var = 0 otherwise NAN problem while computation)
            mean = [data[0][0], data[1][0]]
            var  = [0, 0]

            for i in range(2, len(data)):

                # Start and end index for computation
                index_start = i - window if i - window >= 0 else 0
                index_end   = i - 1 if i - 1 >= 0 else 0

                # Computing mean and variance (much faster using numpy variables)
                mean.append(np.mean(data[index_start:index_end]))
                var.append(np.var(data[index_start:index_end]))
            
            # Adding the new data to dataset
            d[nm] = mean
            if variance:
                d[nv] = var

# Used to compute the instantenous mean and variance of a variable accross multiple datasets
def computeZonalValue(datasets, 
                      variables = ["U100"           , "V100"],
                      names_m   = ["U100_zonal_mean", "V100_zonal_mean"],
                      names_v   = ["U100_zonal_var" , "V100_zonal_var"],
                      variance  = True):

    # Security
    assert len(variables) == len(names_m), "Each variable should have a name for its zonal mean"
    assert len(variables) == len(names_v), "Each variable should have a name for its zonal var"
    assert len(datasets) > 1, "To compute mean and var, at least 2 datasets are needed"

    # Looping over the variables whose mean and var must be computed
    for v, nm, nv in zip(variables, names_m, names_v):

        # Number of samples
        nb_samples = len(datasets[0])

        # Stores all the different values in numpy matrix for efficient computation
        data = np.zeros((nb_samples, len(datasets)))

        # Retreiving all the corresponding data
        for i, d in enumerate(datasets):
            
            # Squeeze is there to remove useless dimension (Ask Victor)
            data[:, i] = np.squeeze(d.loc[: , [v]].to_numpy())

        # Computing mean and variance (much faster using numpy variables)
        mean = np.mean(data, axis = 1) # Axis = 1 to make mean over each row
        var  = np.var(data, axis = 1)

        # Adding new data to all the datasets
        for d in datasets:
            d[nm] = mean
            if variance:
                d[nv] = var

# Used to add the value taken by a given variable over the past samples
def addPastTime(datasets,
                variables = ["U100", "V100"],
                window    = 3):
    #
    # Note from Victor
    # This function was a pain in the ass to make ! Even I, am not sure why it works well :D
    #
    # Security
    assert window > 0, "Window size must be greater than 0 to add past samples"

    # Looping over the datasets
    for d in datasets:

        # Looping over the different columns
        for i, v in enumerate(variables):

            # Retrieving current data
            data = d[[v]].to_numpy()

            # Stores all the past results
            former_data = np.zeros((len(data), window))

            # Looping over the corresponding data
            for j in range(len(data)):

                # Start and end index for retreiving values
                index_start = j - window if j - window >= 0 else 0
                index_end   = j if j - 1 >= 0 else 0
                
                # Retrieve corresponding value
                values = data[index_start:index_end]

                # Fixing case where looking at starting indexes < window size
                if len(values) != window:
                    values = np.append(np.zeros((window - len(values), 1)), values)

                # Placing the data (such that by reading left to right: t - 1, t - 2, t - 3, ...)
                for k, val in enumerate(values):
                        former_data[j][k] = val

            # Addding past results in the dataset
            for t in range(window):
                d[f"{v}_(t-{window - t})"] = former_data[:, t]

# Used to normalize the data of different variables
def normalize(datasets,
              norm_type = "argmax",
              data_type = "column",
              variables = ["U100", "V100"]):
    """
    Documentation :
        - norm_type (str) : argmax, mean
            - Normalize using the argmax or by using the mean and std of the data
        - data_type (str) : column, all
            - Apply the normalization using norm_type computed either on a unique column or all the same columns
    """
    # Security
    assert norm_type in ["argmax", "mean"], "Normalization types = argmax, mean"
    assert data_type in ["column", "all"] , "Data types = column, all"

    # Initialization of the normalization variables
    argmax, mean, std = list(), list(), list()

    # 1 - Computing argmax or mean and std of all datasets
    if data_type == "all":

        # Looping over the different variables to normalize
        for i, v in enumerate(variables):

            # Initialization of the normalization variables
            argmax_list, mean_list, std_list = list(), list(), list()

            # Looping over all the datasets
            for d in datasets:

                # Retrieving currently observed data
                current_data = d[[v]].to_numpy()

                # Retrieving variables
                argmax_list.append(np.max(np.abs(current_data)))
                mean_list.append(np.mean(current_data))
                std_list.append(np.std(current_data))

            # Adding results
            argmax.append(max(argmax_list))
            mean.append(sum(mean_list)/len(mean_list))
            std.append(sum(std_list)/len(std_list))
    
    # 2 - Normalization of the datasets
    for d in datasets:

        # Looping over the different columns
        for i, v in enumerate(variables):
            
            # Case 1 - Mean and std - Column
            if norm_type == "mean" and data_type == "column":
                data         = d[[v]].to_numpy()
                d[v] = (data - np.mean(data))/np.std(data)

            # Case 2 - Mean and std - All
            elif norm_type == "mean" and data_type == "all":
                data         = d[[v]].to_numpy()
                d[v] = (data - mean[i])/std[i]
            
            # Case 3 - Argmax - Column
            elif norm_type == "argmax" and data_type == "column":
                data = d[[v]].to_numpy()
                d[v] = data/np.max(data)

            # Case 4 - Argmax - All
            else:
                data = d[[v]].to_numpy()
                d[v] = data/argmax[i]

# Used to remove specific columns from the dataset
def remove(datasets, variables):

    # Looping over all datasets and variables
    for d in datasets:
        for v in variables:

            # Removing
            d.drop(v, inplace = True, axis = 1)

In [75]:
# -- DATA LOADER -- 
# This class has for purpose to handle the data and make our life easier ! 
#
class dataLoader():
    
    # Initialization of the loader
    def __init__(self, datasets_X, datasets_Y):

        # Stores the original, transformed and final datasets
        self.original_datasets_X    = datasets_X
        self.original_datasets_Y    = datasets_Y
        self.transformed_datasets_X = datasets_X
        self.transformed_datasets_Y = datasets_Y
        self.final_dataset_X        = None
        self.final_dataset_Y        = None

        # Used to know if datasets have been combined or not
        self.isCombined = None

    # Used to display the head of the transformed dataset (first set)
    def showHeadTransformed(self):
        section("Dataset - X - Transformed")
        print(self.transformed_datasets_X[0].head())
        section("Dataset - Y - Transformed")
        print(self.transformed_datasets_Y[0].head())

    # Used to split the final dataset into a train and test set (In test set, values for y are equal to -1)
    def splitTrainTest(self, save = False, save_dir = "new_data"):

        # Security
        assert self.isCombined != None, "You must first use self.finalize"

        # Case 1 - Datasets have been combined all together
        if self.isCombined == True:
            X_train = self.final_dataset_X[self.final_dataset_Y['TARGETVAR'] != -1]
            Y_train = self.final_dataset_Y[self.final_dataset_Y['TARGETVAR'] != -1]
            X_test  = self.final_dataset_X[self.final_dataset_Y['TARGETVAR'] == -1]
            Y_test  = self.final_dataset_Y[self.final_dataset_Y['TARGETVAR'] == -1] # Not useful, I know !

        # Case 2 - Datasets are still separated
        if self.isCombined == False:
            
            X_train, Y_train, X_test, Y_test = list(), list(), list(), list()

            # Looping over all the small datasets
            for x, y in zip(self.final_dataset_X, self.final_dataset_Y):
                X_train.append(x[y['TARGETVAR'] != -1])
                Y_train.append(y[y['TARGETVAR'] != -1])
                X_test.append(x[y['TARGETVAR'] == -1])
                Y_test.append(y[y['TARGETVAR'] == -1])

        # Be careful with the order
        return X_train, X_test, Y_train, Y_test
        
    # Used to perfom final operation on dataset (Combining everything or storing them separately)
    def finalization(self, dataset_type = "combined"):

        # Security
        assert dataset_type in ["combined", "separated"], "The final dataset can either be of type combined or separated"

        # Case 1 - Combining into one big dataset
        if dataset_type == "combined":
            self.final_dataset_X = pd.concat(self.transformed_datasets_X)
            self.final_dataset_Y = pd.concat(self.transformed_datasets_Y)
            self.isCombined = True

        # Case 2 - Separated datasets
        else:
            self.final_dataset_X = self.transformed_datasets_X
            self.final_dataset_Y = self.transformed_datasets_Y
            self.isCombined = False

    # Used to set as final dataset the original dataset
    def original(self, combined = "True"): 
        
        # Setting up for splitting
        self.isCombined = None

        # Copying original dataset
        dX = self.original_datasets_X
        dY = self.original_datasets_Y

        # Updating dataset
        self.transformed_datasets_X = dX
        self.transformed_datasets_Y = dY

    #--------------------------------------------------------------------------------
    #                                    PIPELINES
    #--------------------------------------------------------------------------------
    #
    # List of functions available:
    #
    # computeMeanVariance(datasets, variables = ["U100", "V100"], names_m = ["U100_mean", "V100_mean"], names_v = ["U100_var", "V100_var"],
    #                     window = 100, variance  = True)
    #
    # computeZonalValue(datasets, variables = ["U100", "V100"], names_m = ["U100_zonal_mean", "V100_zonal_mean"], 
    #                   names_v = ["U100_zonal_var" , "V100_zonal_var"], variance  = True)
    # 
    # addPastTime(datasets, variables = ["U100", "V100"], window = 3):
    #
    # normalize(datasets, norm_type = "argmax", data_type = "column", variables = ["U100", "V100"])
    #
    # TYPE 1 - This pipeline is used to create the correlation plots and gain intuition regarding which variable to use
    def pipeline_corellation(self, norm_type = "argmax", data_type = "column"):

        # Definition of the variables to improve
        var      = ["U10", "V10", "U100", "V100"]
        n_aver   = ["U10_mean", "V10_mean", "U100_mean", "V100_mean"]
        n_var    = ["U10_var" , "V10_var", "U100_var" , "V100_var"]
        n_z_aver = ["U10_zonal_mean", "V10_zonal_mean", "U100_zonal_mean", "V100_zonal_mean"]
        n_z_var  = ["U10_zonal_var" , "V10_zonal_var", "U100_zonal_var" , "V100_zonal_var"]

        # Copying original dataset
        dX = self.original_datasets_X
        dY = self.original_datasets_Y

        # Applying transformations
        normalize(dX, norm_type = norm_type, data_type = data_type, variables = var)
        computeMeanVariance(dX, variables = var, names_m = n_aver,   names_v = n_var, window = 24, variance = True)
        computeZonalValue(dX, variables = var, names_m = n_z_aver, names_v = n_z_var, variance = True)
        addPastTime(dX, variables = var, window = 3)

        # Updating dataset
        self.transformed_datasets_X = dX
        self.transformed_datasets_Y = dY

        # Making sure one has to finalize again
        self.isCombined = None

    # TYPE 2 - Normalizing all the data, removing year and day
    def pipeline_1(self):

        # Definition of the variables to improve
        var      = ["U10", "V10", "U100", "V100"]

        # Copying original dataset
        dX = copy.deepcopy(self.original_datasets_X)
        dY = copy.deepcopy(self.original_datasets_Y)

        # Applying transformations
        normalize(dX, norm_type = "argmax", data_type = "column", variables = var)
        
        # Removing useless information
        #remove(dX, ["Day"])
        
        # Updating dataset
        self.transformed_datasets_X = dX
        self.transformed_datasets_Y = dY

        # Making sure one has to finalize again
        self.isCombined = None

    # TYPE 2 - Normalizing all the data, removing year and day
    def pipeline_2(self, variance = True):

        # Copying original dataset
        dX = copy.deepcopy(self.original_datasets_X)
        dY = copy.deepcopy(self.original_datasets_Y)

        # Applying transformations
        normalize(dX, norm_type = "argmax", data_type = "column")
        
        # Removing useless information
        #remove(dX, ["Day"])
        computeMeanVariance(dX, window = 24, variance  = variance)
        
        # Updating dataset
        self.transformed_datasets_X = dX
        self.transformed_datasets_Y = dY

        # Making sure one has to finalize again
        self.isCombined = None


In [ ]:
# -- GAINING INSIGHTS (1) - ARGMAX COLUMN -- 
#
# Treshold value for removing correlation
tresh_corr = 0.3

# Initialization of the loader
loader_1 = dataLoader(dataset_original_X, dataset_original_Y)

# Aplying the first prototype of pipeline (Mean value on 30 days)
loader_1.pipeline_corellation(norm_type = "argmax", data_type = "column")

# Combine all the small datasets into a big one
loader_1.finalization(dataset_type = "combined")

# Retreives the train and test set (in Pandas frame)
data_X_1, _, _, _ = loader_1.splitTrainTest()

# -- Correlation matrix -- 
corr_1             = data_X_1.corr()
corr_1[np.abs(corr_1) < tresh_corr] = 0
sns.set(rc={'figure.figsize':(25, 20)})
sns.heatmap(corr_1, cmap = "Blues", annot = True)

In [ ]:
# -- GAINING INSIGHTS (2) - ARGMAX ALL -- 
#
# Treshold value for removing correlation
tresh_corr = 0.3

# Initialization of the loader
loader_2 = dataLoader(dataset_original_X, dataset_original_Y)

# Aplying the first prototype of pipeline (Mean value on 30 days)
loader_2.pipeline_corellation(norm_type = "argmax", data_type = "all")

# Combine all the small datasets into a big one
loader_2.finalization(dataset_type = "combined")

# Retreives the train and test set (in Pandas frame)
data_X_2, _, _, _ = loader_2.splitTrainTest()

# -- Correlation matrix -- 
corr_2             = data_X_2.corr()
corr_2[np.abs(corr_2) < tresh_corr] = 0
sns.set(rc={'figure.figsize':(25, 20)})
sns.heatmap(corr_2, cmap = "YlGnBu", annot = True)

In [ ]:
# -- GAINING INSIGHTS (3) - MEAN COLUMN -- 
#
# Treshold value for removing correlation
tresh_corr = 0.3

# Initialization of the loader
loader_3 = dataLoader(dataset_original_X, dataset_original_Y)

# Aplying the first prototype of pipeline (Mean value on 30 days)
loader_3.pipeline_corellation(norm_type = "mean", data_type = "column")

# Combine all the small datasets into a big one
loader_3.finalization(dataset_type = "combined")

# Retreives the train and test set (in Pandas frame)
data_X_3, _, _, _ = loader_3.splitTrainTest()

# -- Correlation matrix -- 
corr_3             = data_X_3.corr()
corr_3[np.abs(corr_3) < tresh_corr] = 0
sns.set(rc={'figure.figsize':(25, 20)})
sns.heatmap(corr_3, cmap = "BuPu", annot = True)

In [ ]:
# -- GAINING INSIGHTS (4) - MEAN ALL -- 
#
# Treshold value for removing correlation
tresh_corr = 0.3

# Initialization of the loader
loader_4 = dataLoader(dataset_original_X, dataset_original_Y)

# Aplying the first prototype of pipeline (Mean value on 30 days)
loader_4.pipeline_corellation(norm_type = "mean", data_type = "all")

# Combine all the small datasets into a big one
loader_4.finalization(dataset_type = "combined")

# Retreives the train and test set (in Pandas frame)
data_X_4, _, _, _ = loader_4.splitTrainTest()

# -- Correlation matrix -- 
corr_4             = data_X_4.corr()
corr_4[np.abs(corr_4) < tresh_corr] = 0
sns.set(rc={'figure.figsize':(25, 20)})
sns.heatmap(corr_4, cmap = "Greens", annot = True)

In [ ]:
# -- CORRELATION MATRIX COMPARISON --
corr_21 = corr_2 - corr_1
sns.set(rc={'figure.figsize':(25, 20)})
sns.heatmap(corr_21, cmap = "Greens", annot = True)

plt.figure()
corr_23 = corr_2 - corr_3
sns.set(rc={'figure.figsize':(25, 20)})
sns.heatmap(corr_23, cmap = "BuPu", annot = True)

plt.figure()
corr_24 = corr_2 - corr_4
sns.set(rc={'figure.figsize':(25, 20)})
sns.heatmap(corr_24, cmap = "YlGnBu", annot = True)

plt.figure()
corr_31 = corr_3 - corr_1
corr_31[np.abs(corr_31) < 0.2] = 0
sns.set(rc={'figure.figsize':(25, 20)})
sns.heatmap(corr_31, cmap = "BuPu", annot = True)

[comment]: <> (Section)
<hr style="color:#a4342d;"></hr>
<p align="center">
    <b style="font-size:1.5vw; color:#a4342d;">
    Model - Training & Testing
    </b>
</p>
<hr style="color:#a4342d;"></hr>

[comment]: <> (Description)
<p align="justify">
    In this section, one will be able to explore further the dataset ! First, one needs to create a train and test set. Then, it is interesitng to look for correlations, new variables and possible improvements to the current dataset. All the new datasets will be save in the datafolder and ready to use by our different models ! The functions available are:
</p>

In [76]:
# -- Creation of datasets

# 1 - Original data
loader_0 = dataLoader(dataset_original_X, dataset_original_Y)
loader_0.original()
loader_0.finalization()
data_X0, submit_X0, data_Y0, submit_Y0 = loader_0.splitTrainTest()

print(data_X0.head())

# 2 - Different version
loader_1 = dataLoader(dataset_original_X, dataset_original_Y)
loader_1.pipeline_1()
loader_1.finalization()
data_X1, submit_X1, data_Y1, submit_Y1 = loader_1.splitTrainTest()

print(data_X1.head())

loader_2 = dataLoader(dataset_original_X, dataset_original_Y)
loader_2.pipeline_2(variance = False)
loader_2.finalization()
data_X2, submit_X2, data_Y2, submit_Y2 = loader_2.splitTrainTest()

print(data_X2.head())

loader_3 = dataLoader(dataset_original_X, dataset_original_Y)
loader_3.pipeline_2(variance = True)
loader_3.finalization()
data_X3, submit_X3, data_Y3, submit_Y3 = loader_3.splitTrainTest()

print(data_X3.head())


   ZONEID       U10       V10      U100      V100  Day  Month  Year  Hour
0       1  2.124600 -2.681966  2.864280 -3.666076    1      1  2012     1
1       1  2.521695 -1.796960  3.344859 -2.464761    1      1  2012     2
2       1  2.672210 -0.822516  3.508448 -1.214093    1      1  2012     3
3       1  2.457504 -0.143642  3.215233 -0.355546    1      1  2012     4
4       1  2.245898  0.389576  2.957678  0.332701    1      1  2012     5
   ZONEID       U10       V10      U100      V100  Month  Year  Hour
0       1  0.191112 -0.282051  0.168602 -0.256111      1  2012     1
1       1  0.226832 -0.188979  0.196891 -0.172187      1  2012     2
2       1  0.240371 -0.086501  0.206520 -0.084816      1  2012     3
3       1  0.221058 -0.015106  0.189260 -0.024838      1  2012     4
4       1  0.202023  0.040970  0.174100  0.023242      1  2012     5
   ZONEID       U10       V10      U100      V100  Month  Year  Hour  \
0       1  2.124600 -2.681966  0.168602 -0.256111      1  2012     1  

In [77]:
# -- Training and experimenting --
#
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble  import RandomForestRegressor

# Contains all the datasets
data_X_tot = [data_X0, data_X1, data_X2, data_X3]
data_Y_tot = [data_Y0, data_Y1, data_Y2, data_Y3]

index = 0

# Testing models on all the datasets and comparing them
for X, Y in zip(data_X_tot, data_Y_tot):

    # Creation of the sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 69)

    # Initialization of the model
    model_1 = RandomForestRegressor(max_depth = 20, n_estimators = 50)
    model_2 = KNeighborsRegressor(n_neighbors = 20)

    # Fitting the data
    print("Fitting...")
    model_1.fit(X_train, y_train)
    print("Fitted - RF")
    model_2.fit(X_train, y_train)
    print("Fitted - RF, KNN")

    # Computing accuracy
    print("Computing accuracy...")
    accuracy_1 = model_1.score(X_test, y_test)
    accuracy_2 = model_2.score(X_test, y_test)

    # Results
    print(f"KNN - Accuracy - Dataset n°{index}= {accuracy_1}")
    print(f"RF  - Accuracy - Dataset n°{index}= {accuracy_2}")
    index += 1




Fitting...
Fitted - RF
Fitted - RF, KNN
Computing accuracy...
KNN - Accuracy - Dataset n°0= 0.8377532991030915
RF  - Accuracy - Dataset n°0= 0.8636027853314285
Fitting...
Fitted - RF
Fitted - RF, KNN
Computing accuracy...
KNN - Accuracy - Dataset n°1= 0.7984708045246564
RF  - Accuracy - Dataset n°1= 0.7375874745999287
Fitting...
Fitted - RF
Fitted - RF, KNN
Computing accuracy...
KNN - Accuracy - Dataset n°2= 0.8357548959048624
RF  - Accuracy - Dataset n°2= 0.7436508119431772
Fitting...
Fitted - RF
Fitted - RF, KNN
Computing accuracy...
KNN - Accuracy - Dataset n°3= 0.8398329826109148
RF  - Accuracy - Dataset n°3= 0.7436819866989951
